In [ ]:
import numpy as np
import mne # docu: https://mne.tools/stable/overview/index.html
import pandas as pd
import matplotlib.pyplot as plt

from PCIst import pci_st # https://github.com/renzocom/PCIst

from matplotlib import rcParams
rcParams['figure.figsize'] = 15,6

import sys
sys.path.append('../')
from src import data

%matplotlib widget

In [ ]:
folder = 'external/nieus_tms_eeg_ebrains/derivatives/epochs/'
subject = 'sub-01'

eeg_epoched_data = np.load(data.path(folder+f'{subject}/eeg/{subject}_task-tmseeg_run-01_epochs.npy'))
metadata = pd.read_csv(data.path(folder+f'{subject}/eeg/{subject}_task-tmseeg_run-01_channels.tsv'), sep = '\t')
montage = mne.channels.read_custom_montage(data.path(folder+f'{subject}/eeg/{subject}_task-tmseeg_electrodes.tsv'))
metadata.head()

In [ ]:
epochs_info = mne.create_info(list(metadata.name), metadata.iloc[0].sampling_frequency)
epochs = mne.EpochsArray(eeg_epoched_data,epochs_info)
epochs.set_channel_types({m:'eeg' for m in metadata.name})
epochs.set_montage(montage)
epochs.average("all").plot();

In [ ]:
epochs_data = epochs.get_data() # (n_epochs, n_channels, n_times), dalo by se rovnou z načteného vstupu, ale to bych neměla hezký obrázek výše
epochs_data_averaged = np.mean(epochs_data,axis=0)

In [ ]:
par = {'baseline_window':(0,0.35), 'response_window':(0.4,0.8), 'k':1.2, 'min_snr':1.1, 'max_var':98, 'embed':False,'n_steps':100} # 
pci_st_result = pci_st.calc_PCIst(epochs_data_averaged, epochs.times, full_return=True, **par)

"""
 'PCI': final PCI value,
 'dNST': list containing component wise PCIst value (∆NSTn),
 'n_dims': number of components,
 'D_base': base distance matrix,
 'D_resp': response distance matrix,
 'T_base': base transition matrix,
 'T_resp': response transition matrix,
 'thresholds': 2D array (n_steps, n_dims) - n_steps thresholds for each dimension,
 'NST_diff': 2D array (n_steps, n_dims) - n_steps values based on n_steps thresholds for each dimension,
 'NST_resp': 2D array (n_steps, n_dims) - n_steps values based on n_steps thresholds for each dimension,
 'NST_base': 2D array (n_steps, n_dims) - n_steps values based on n_steps thresholds for each dimension,
 'max_thresholds': threshold with maximum NST_diff for each dimension,
 'signal_evk': ,
 'times': ,
 'signal_svd': ,
 'eigenvalues': ,
 'var_exp': ,
 'snrs': signal to noise ratio for each component
"""

pci_st_result

In [ ]:
for d in range(pci_st_result["n_dims"]):

    plt.figure()
    plt.title(f"Componet {d}")
    plt.plot(pci_st_result["times"],pci_st_result["signal_svd"][d])
    plt.axvline(x=0.4,color="k")
    plt.show()

    fig, (ax0,ax1) = plt.subplots(1, 2, sharey=True)
    

    min_base = np.min(pci_st_result['D_base'][d])
    max_base = np.max(pci_st_result['D_base'][d])
    min_resp = np.min(pci_st_result['D_resp'][d])
    max_resp = np.max(pci_st_result['D_resp'][d])
    minimum = min(min_base,min_resp)
    maximum = max(max_base,max_resp)

    ax0.imshow(pci_st_result['D_base'][d],vmin=minimum, vmax=maximum, norm="symlog", cmap='nipy_spectral')
    ax1.imshow(pci_st_result['D_resp'][d],vmin=minimum, vmax=maximum, norm="symlog", cmap='nipy_spectral')

    ax0.set_title('base')
    ax1.set_title('response')

    plt.show()

    plt.figure()
    plt.axvline(x = pci_st_result["max_thresholds"][d], color="y", label = 'e^*')
    plt.plot(pci_st_result['thresholds'][:,d], pci_st_result['NST_base'][:,d], 'black', label='NST_base')
    plt.plot(pci_st_result['thresholds'][:,d], pci_st_result['NST_resp'][:,d], 'b', label='NST_resp')
    plt.plot(pci_st_result['thresholds'][:,d], pci_st_result['NST_diff'][:,d], 'r', label='NST_diff')

    plt.show()

    fig, (ax0,ax1) = plt.subplots(1, 2, sharey=True)

    im0 = ax0.imshow(pci_st_result['T_base'][d])
    im1 = ax1.imshow(pci_st_result['T_resp'][d])

    ax0.set_title('base thresholded')
    ax1.set_title('response thresholded')

    plt.show()

In [ ]:
pci_st_result["PCI"]